# Perceptrón Multicapa

Como modelo simple para ir aprendiendo el uso de PyTorch vamos a crear un perceptrón multicapa y entrenarlo con los datos en bruto.

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import pandas as pd
import torch 
import matplotlib.pyplot as plt

from utils import *

In [3]:
PATH = "./datos/"
TRAIN_FILE = "exoTrain.csv"
TEST_FILE = "exoTest.csv"

In [4]:
#leemos los datos de ambos datasets
df_train_raw = pd.read_csv(f'{PATH}{TRAIN_FILE}', low_memory=False)
#df_test_raw = pd.read_csv(f'{PATH}{TEST_FILE}', low_memory=False)

In [5]:
train_x, train_y = generate_x_y_df(df_train_raw)
train_y = train_y.add(-1) #pasamos de 1-2 a 0-1

In [6]:
train_x.shape, train_y.shape

((5087, 3197), (5087,))

In [7]:
#definicion del modelo
class Perceptron(torch.nn.Module):
    #definimos la estructura de la red
    #capa de entrada x con activacion relu
    #capa oculta con activacion relu
    #capa de salida con activacion softmax
    def __init__(self, n_features, n_hidden):
        super(Perceptron, self).__init__()
        self.fully_connected_1 = torch.nn.Linear(n_features, n_hidden)
        self.fully_connected_2 = torch.nn.Linear(n_hidden, n_hidden)
        self.fully_connected_3 = torch.nn.Linear(n_hidden, 2)
        self.relu = torch.nn.ReLU()
        self.softmax = torch.nn.Softmax(dim = 1)
        #self.softmax = torch.nn.Sigmoid()
        
    #definimos el flujo 
    def forward(self, x):
        x = self.relu(self.fully_connected_1(x))
        x = self.relu(self.fully_connected_2(x))
        x = self.softmax(self.fully_connected_3(x))
        #x = self.fully_connected_3(x)
        return x

In [20]:
learning_rate = 0.01
epochs = 50
train_x_tensor = torch.tensor(train_x.values).float()
train_y_tensor = torch.tensor(train_y.values)

In [21]:
#instanciamos nuestro modelo
modelo = Perceptron(n_features = train_x.shape[1], n_hidden = 300) 

In [22]:
optimizer = torch.optim.SGD(modelo.parameters(), lr = learning_rate)
loss_function = torch.nn.CrossEntropyLoss()

In [23]:
for epoch in range(epochs):
    #limpiamos los gradientes
    optimizer.zero_grad()
    #propagamos la entrada hacia delante
    predictions = modelo(train_x_tensor)
    #calculamos la perdida
    loss = loss_function(predictions.squeeze(), train_y_tensor)
    #ejecutamos back-propagation
    loss.backward()
    #actualizamos los pesos
    optimizer.step()
    #imprimimos
    if epoch % 10 == 0:
        print('Epoch: {} loss: {}'.format(epoch, loss.item()))
        
print('Epoch: {} loss: {}'.format(epochs, loss.item()))

Epoch: 0 loss: 0.7285034656524658
Epoch: 10 loss: 0.3257288336753845
Epoch: 20 loss: 0.3246678113937378
Epoch: 30 loss: 0.32399091124534607
Epoch: 40 loss: 0.32351645827293396
Epoch: 50 loss: 0.3231947124004364


In [26]:
predictions

tensor([[1.0000e+00, 2.1440e-43],
        [1.0000e+00, 3.5632e-15],
        [1.0000e+00, 0.0000e+00],
        ...,
        [1.0000e+00, 2.4837e-15],
        [9.6151e-01, 3.8488e-02],
        [1.0000e+00, 1.5333e-13]], grad_fn=<SoftmaxBackward>)

Como sospechabamos, sin hacer ningun tratamiento previo a los datos, el modelo solamente devuelve que no existe exoplaneta con certeza absoluta en practicamente todos los casos.